## Binary Quantization

* 벡터의 각 차원을 1비트로 변환하여 메모리 절약
* 양수는 1, 음수는 0으로 변환하여 방향성만 유지
* 메모리 효율성과 처리 속도를 높이지만, 정확도가 떨어질 수 있음
* similarity search 진행 시, oversampling 후 re-ranking 하는 방식으로 보완 가능

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import time

f = h5py.File('openai_embedding_output.hdf5','r')
distances = f['distances']
neighbors = f['neighbors']
test = f['test']
train = f['train']

In [17]:
import numpy as np
from scipy.spatial.distance import hamming, cosine

### example code ###

class BinaryQuantization:
    def __init__(self, vectors):
        self.vectors = self.binary_quantization(vectors)

    def binary_quantization(self, vectors):
        return np.where(vectors >= 0, 1, 0)

    def search(self, query, k):
        query_bin = self.binary_quantization(query)
        distances = np.array([hamming(query_bin, vec_bin) for vec_bin in self.vectors])
        closest_index = np.argsort(distances)[:k]
        return [(idx, distances[idx]) for idx in closest_index]


In [18]:
bq = BinaryQuantization(train[:])

In [19]:
bq.search(test[0], 5)

[(3477, 0.2571614583333333),
 (5303, 0.2643229166666667),
 (534, 0.26953125),
 (4134, 0.2734375),
 (7981, 0.2799479166666667)]

In [20]:
[(n, d) for d, n in zip(distances[0][:5], neighbors[0][:5])]

[(3477, 0.1564233992352113),
 (7981, 0.17076890207250994),
 (5303, 0.17926627397536643),
 (4134, 0.18096811657953815),
 (534, 0.18518513441085238)]

In [21]:
## search speed and accuracy
accuracies_bq = []
times = []

for i in range(len(test)):
    start = time.time()
    search_result = bq.search(test[i], 5)
    times.append(time.time() - start)

    acc = len(set(neighbors[i][:5]) & set([idx for idx, _ in search_result])) / 5
    accuracies_bq.append(acc)

print(f"accuracy : {np.mean(accuracies_bq)}, qps : {1 / np.mean(times)}")


## BQ만으로는 성능 부족 => over-filtering 후 re-ranking 필요

accuracy : 0.6779999999999998, qps : 6.42656478843111


## Homework (2024.11.20 (수) 11:59 PM 까지)

*   해당 파일에 작성하여 제출
*   파일명: homework_이름_학번.ipynb




1. IVF, HNSW에 binary quantization 적용
    *   index build or search (cf. IVF-PQ, HNSW-PQ)
    *   구현 방식에 대한 간략한 설명


In [ ]:
class IVFBQ():
    ...

In [30]:

class HNSWBQ():
    ...



2. 적용 이후 기존의 IVF, HNSW와 성능 비교 분석 (recall, qps, 방문 노드 수 등)
    *   정확도 저하 시 oversampling 및 re-ranking 진행
    *   BQ를 적용함으로써 얻는 이익과 어떤 상황에서 BQ를 사용하는 것이 적절할지 서술


